# 人工干预 Human in the loop

人工干预（Human-in-the-loop）中间件允许我们为代理工具调用添加人工监督。当模型提出一个可能需要审核的操作时——例如写入文件或执行 SQL——中间件可以暂停执行并等待人工决策。

该中间件通过将每个工具调用与可配置的策略进行比对来实现这一点。如果需要人工干预，中间件会发出一个中断信号，暂停执行。执行状态将通过 LangGraph 的持久化层保存，因此可以安全地暂停并稍后恢复执行。

然后由人工决策决定下一步操作：该操作可以原样批准（approve）、在运行前进行修改（edit），或被拒绝并提供反馈（reject）。


In [9]:
# 配置开发环境
import os
from dotenv import load_dotenv
from pathlib import Path


load_dotenv()
root_path = Path.cwd().parent.parent

str(root_path), os.getenv("OPENAI_BASE_URL"), os.getenv("OPENAI_API_KEY")


('/home/zht/projects/Agent',
 'https://api.openai-proxy.org/v1',
 'sk-A7irm5LFwUUuMHPQ58V5qnygQQEYOGm4J8bfiUo5jm8N0lai')

我们首先创建两个 tool 用于后续演示：



In [11]:
from langchain.tools import tool

@tool
def get_user_info(user_id: str) -> str:
    """ 获取用户信息 """
    return f"用户 {user_id} 的信息"


@tool 
def write_file(file_path: str, content: str) -> str:
    """ 写入文件 """
    return f"文件 {file_path} 已写入, 内容为 {content}"



显然此时写入文件的风险比较大一些，所以我们在创建 agent 的时候，要为该工具的执行添加人工干预。

下面先创建一个代理：


In [ ]:
from langchain.chat_models import init_chat_model
from langchain.agents import create_agent
from langchain_core.messages import HumanMessage


agent = create_agent(
    model = init_chat_model("openai:gpt-4o-mini"),
    tools = [get_user_info, write_file],
)

resp = agent.invoke(
    input = {
        "messages": [
            HumanMessage(content="请根据用户 id 获取用户信息：iris_10001"),
            HumanMessage(content="请根据用户信息写入文件: user_id: iris_10001 content: age: 20"),
        ]
    }
)

resp

{'messages': [HumanMessage(content='请根据用户 id 获取用户信息：iris_10001', additional_kwargs={}, response_metadata={}, id='26c0f916-ec50-413d-a6ac-d7b34b86e9f4'),
  HumanMessage(content='请根据用户信息写入文件: user_id: iris_10001 content: age: 20', additional_kwargs={}, response_metadata={}, id='4cb3ab8e-dccd-4fae-a978-f76b628bb2c7'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 70, 'prompt_tokens': 103, 'total_tokens': 173, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_efad92c60b', 'id': 'chatcmpl-Ch5kww2vIukC8OisSaG4ycnMB7eqb', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--d07e6efb-b876-4705-b7aa-dedc297d5f24-0', tool_calls=[{'name': 'get_user_info', 'args': {'user_id

在没有人工干预的情况下，代理通常会自动调用工具，所以我们需要通过 HITL 中间件为其设置中断：


In [29]:
from langchain.agents.middleware import HumanInTheLoopMiddleware 
from langgraph.checkpoint.memory import InMemorySaver 


# 定义 HITL 中间件
human_in_the_loop = HumanInTheLoopMiddleware(
    interrupt_on = {
        "write_file": True,
        "get_user_info": False,
    },
    description_prefix="Tool execution pending approval",
)

agent = create_agent(
    model = init_chat_model("openai:gpt-4o-mini"),
    tools = [get_user_info, write_file],
    middleware=[human_in_the_loop],
    checkpointer=InMemorySaver(),
)


接下来我们演示一下，加入 HITL 之后，工具调用会产生什么变化：


首先尝试调用获取信息的工具：


In [30]:
config = {"configurable": {"thread_id": "t_10086"}} 

resp = agent.invoke(
    input = {
        "messages": [
            HumanMessage(content="请根据用户 id 获取用户信息：iris_10001"),
        ]
    },
    config = config
)

resp

{'messages': [HumanMessage(content='请根据用户 id 获取用户信息：iris_10001', additional_kwargs={}, response_metadata={}, id='7e251726-4923-4b92-ac45-15ed1f7855a8'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 78, 'total_tokens': 98, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_efad92c60b', 'id': 'chatcmpl-Ch66LlhV5Mi8RhplQrQRcpC1ysgm7', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--4ce36ab4-7924-472d-9c78-bc01719b56f8-0', tool_calls=[{'name': 'get_user_info', 'args': {'user_id': 'iris_10001'}, 'id': 'call_oS6LsoGgkSeVwnKokBPgeunV', 'type': 'tool_call'}], usage_metadata={'input_tokens': 78, 'output_tokens': 20, 'total_tokens': 98, 'input_to

然后尝试调用需要人工审核的工具：


In [31]:
resp = agent.invoke(
    input = {
        "messages": [
            HumanMessage(content="写入用户信息：user_id: iris_10001 content: age: 20"),
        ]
    },
    config = config
)

resp

{'messages': [HumanMessage(content='请根据用户 id 获取用户信息：iris_10001', additional_kwargs={}, response_metadata={}, id='7e251726-4923-4b92-ac45-15ed1f7855a8'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 78, 'total_tokens': 98, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_efad92c60b', 'id': 'chatcmpl-Ch66LlhV5Mi8RhplQrQRcpC1ysgm7', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--4ce36ab4-7924-472d-9c78-bc01719b56f8-0', tool_calls=[{'name': 'get_user_info', 'args': {'user_id': 'iris_10001'}, 'id': 'call_oS6LsoGgkSeVwnKokBPgeunV', 'type': 'tool_call'}], usage_metadata={'input_tokens': 78, 'output_tokens': 20, 'total_tokens': 98, 'input_to

显然，这次 resp 的最后一条内容不是 AIMessage，而是变成了 `__interrupt__` 字段，其中列出了需要审核的操作。

我们可以将这些操作显示出来，并让人工审核对其进行决策。


In [ ]:
from langgraph.types import Command

# 获取所有中断
if resp.get("__interrupt__"):
    hitl_req = resp["__interrupt__"][0].value

    action_requests = hitl_req["action_requests"]
    review_configs = hitl_req["review_configs"]  # 里面写着允许的决策类型

    # 简单打印一下要人工确认的操作
    for action in action_requests:
        print("需要人工确认的工具:", action["name"])
        print("参数:", action["args"])
else:
    print("没有触发中断，直接看 resp['messages'] 即可")


需要人工确认的工具: write_file
参数: {'file_path': 'iris_10001_info.txt', 'content': 'user_id: iris_10001\nage: 20'}


接下来我们在新的线程中调用模型并处理中断：


In [50]:
from langgraph.types import Command


config = {"configurable": {"thread_id": "114514"}}

resp = agent.invoke(
    input = {
        "messages": [
            HumanMessage(content="写入用户信息：user_id: iris_10001 content: age: 20"),
        ]
    },
    config = config
)


resp


{'messages': [HumanMessage(content='写入用户信息：user_id: iris_10001 content: age: 20', additional_kwargs={}, response_metadata={}, id='11b05285-65ac-4f73-a221-c68198873176'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 84, 'total_tokens': 122, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_efad92c60b', 'id': 'chatcmpl-Ch6NXLrULV2Y9zOw1zWNxvdWNEXd1', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--6b9a9873-b1ae-4cce-bc8b-9462b2265bc1-0', tool_calls=[{'name': 'write_file', 'args': {'file_path': 'user_info_iris_10001.txt', 'content': 'user_id: iris_10001\nage: 20'}, 'id': 'call_c0FYL4BzdDPYpCVDOBWh1ZuJ', 'type': 'tool_call'}], usage_metada

In [ ]:
# # 通过中断
# agent.invoke(
#     Command( 
#         resume={"decisions": [{"type": "approve"}]}  # or "edit", "reject"
#     ), 
#     config=config # Same thread ID to resume the paused conversation
# )

In [51]:
from pprint import pprint
pprint(resp["__interrupt__"])


[Interrupt(value={'action_requests': [{'args': {'content': 'user_id: '
                                                           'iris_10001\n'
                                                           'age: 20',
                                                'file_path': 'user_info_iris_10001.txt'},
                                       'description': 'Tool execution pending '
                                                      'approval\n'
                                                      '\n'
                                                      'Tool: write_file\n'
                                                      "Args: {'file_path': "
                                                      "'user_info_iris_10001.txt', "
                                                      "'content': 'user_id: "
                                                      "iris_10001\\nage: 20'}",
                                       'name': 'write_file'}],
                  'review_confi

In [ ]:
# # 编辑中断
# agent.invoke(
#     Command(
#         resume={
#             "decisions": [
#                 {
#                     "type": "edit",
#                     "edited_action": {
#                         "name": "write_file",  # 填写 tool_name
#                         "args": {"file_path": "user_info_iris_10001.txt", "content": "user_id: iris_10001"},
#                     }
#                 }
#             ]
#         }
#     ),
#     config=config  # Same thread ID to resume the paused conversation
# )

{'messages': [HumanMessage(content='写入用户信息：user_id: iris_10001 content: age: 20', additional_kwargs={}, response_metadata={}, id='11b05285-65ac-4f73-a221-c68198873176'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 84, 'total_tokens': 122, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_efad92c60b', 'id': 'chatcmpl-Ch6NXLrULV2Y9zOw1zWNxvdWNEXd1', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--6b9a9873-b1ae-4cce-bc8b-9462b2265bc1-0', tool_calls=[{'name': 'write_file', 'args': {'file_path': 'user_info_iris_10001.txt', 'content': 'user_id: iris_10001\nage: 20'}, 'id': 'call_c0FYL4BzdDPYpCVDOBWh1ZuJ', 'type': 'tool_call'}], usage_metada

In [52]:
# 拒绝中断

agent.invoke(
    Command(
        resume={
            "decisions": [
                {
                    "type": "reject",
                    "message": "添加的信息不合规，拒绝实施。",
                }
            ]
        }
    ),
    config=config  
)

{'messages': [HumanMessage(content='写入用户信息：user_id: iris_10001 content: age: 20', additional_kwargs={}, response_metadata={}, id='11b05285-65ac-4f73-a221-c68198873176'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 84, 'total_tokens': 122, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_efad92c60b', 'id': 'chatcmpl-Ch6NXLrULV2Y9zOw1zWNxvdWNEXd1', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--6b9a9873-b1ae-4cce-bc8b-9462b2265bc1-0', tool_calls=[{'name': 'write_file', 'args': {'file_path': 'user_info_iris_10001.txt', 'content': 'user_id: iris_10001\nage: 20'}, 'id': 'call_c0FYL4BzdDPYpCVDOBWh1ZuJ', 'type': 'tool_call'}], usage_metada